<hr />

### NoteBookの見方
コード以外の情報 
<span >白/黒での記載は速習Symbol</span>  
<span style="color:red">赤色での記載は補足情報</span>  
<span >コード内で🌟マークがある場所は自分の情報に書き換えが必要</span>

<hr />

<span style="color:red">

# 環境構築
</span>
<span style="color:red">

## 1.Symbol SDKの読み込み </span>

In [ ]:
(script = document.createElement("script")).src = "https://xembook.github.io/nem2-browserify/symbol-sdk-pack-2.0.3.js";
document.getElementsByTagName("head")[0].appendChild(script);

<span style="color:red">

## 2.Symbol用の共通設定 </span>

In [ ]:
NODE = 'https://sym-test-03.opening-line.jp:3001';
sym = require("/node_modules/symbol-sdk");
repo = new sym.RepositoryFactoryHttp(NODE);
txRepo = repo.createTransactionRepository();
mosaicRepo = repo.createMosaicRepository();
accountRepo = repo.createAccountRepository();
(async () => {
  networkType = await repo.getNetworkType().toPromise();
  generationHash = await repo.getGenerationHash().toPromise();
  epochAdjustment = await repo.getEpochAdjustment().toPromise();
})();

function clog(signedTx){
    console.log(NODE + "/transactionStatus/" + signedTx.hash);
    console.log(NODE + "/transactions/confirmed/" + signedTx.hash);
    console.log("https://symbol.fyi/transactions/" + signedTx.hash);
    console.log("https://testnet.symbol.fyi/transactions/" + signedTx.hash);
}

<span style="color:red">

## 3.aliceアカウントのリストア </span>

In [2]:
alice = sym.Account.createFromPrivateKey(
    "1E9139CC1580B4AED6A1FE110085281D4982ED0D89CE07F3380EB83069B1****", //🌟ここに3章で作成した秘密鍵を入力
    networkType
);

![image](https://user-images.githubusercontent.com/47712051/232868663-dfef36fd-ce36-4631-bf3e-d0d3d8b454f6.png)

# 4.トランザクション
ブロックチェーン上のデータ更新はトランザクションをネットワークにアナウンスすることによって行います。

## 4.1 トランザクションのライフサイクル

トランザクションを作成してから、改ざんが困難なデータとなるまでを順に説明します。

- トランザクション作成
  - ブロックチェーンが受理できるフォーマットでトランザクションを作成します。
- 署名
  - アカウントの秘密鍵でトランザクションを署名します。
- アナウンス
  - 任意のノードに署名済みトランザクションを通知します。
- 未承認トランザクション
  - ノードに受理されたトランザクションは、未承認トランザクションとして全ノードに伝播します
    - トランザクションに設定した最大手数料が、各ノード毎に設定されている最低手数料を満たさない場合はそのノードへは伝播しません。
- 承認済みトランザクション
  - 約30秒に1度ごとに生成されるブロックに未承認トランザクションが取り込まれると、承認済みトランザクションとなります。
- ロールバック
  - ノード間の合意に達することができずロールバックされたブロックに含まれていたトランザクションは、未承認トランザクションに差し戻されます。
    - 有効期限切れや、キャッシュからあふれたトランザクションは切り捨てられます。
- ファイナライズ
  - 投票ノードによるファイナライズプロセスによりブロックが確定するとトランザクションはロールバック不可なデータとして扱うことができます。

### ブロックとは

ブロックは約30秒ごとに生成され、高い手数料を支払ったトランザクションから優先に取り込まれ、ブロック単位で他のノードと同期します。
同期に失敗するとロールバックして、ネットワークが全体で合意が取れるまでこの作業を繰り返します。

## 4.2 トランザクション作成

まずは最も基本的な転送トランザクションを作成してみます。

### Bobへの転送トランザクション

送信先のBobアドレスを作成しておきます。

In [3]:
bob = sym.Account.generateNewAccount(networkType);
console.log(bob.address);

トランザクションを作成します。


In [26]:
tx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment), //Deadline:有効期限
    sym.Address.createFromRawAddress("TDWBA6L3CZ6VTZAZPAISL3RWM5VKMHM6J6IM3LY"), 
    [],
    sym.PlainMessage.create("Hello Symbol!"), //メッセージ
    networkType //テストネット・メインネット区分
).setMaxFee(100); //手数料

各設定項目について説明します。

#### 有効期限
sdkではデフォルトで2時間後に設定されます。
最大6時間まで指定可能です。
```js
sym.Deadline.create(epochAdjustment,6)
```

#### メッセージ
トランザクションに最大1023バイトのメッセージを添付することができます。
バイナリデータであってもrawdataとして送信することが可能です。

##### 空メッセージ
```js
sym.EmptyMessage
```

##### 平文メッセージ
```js
sym.PlainMessage.create("Hello Symbol!")
```

##### 暗号文メッセージ
```js
sym.EncryptedMessage('294C8979156C0D941270BAC191F7C689E93371EDBC36ADD8B920CF494012A97BA2D1A3759F9A6D55D5957E9D');
```

EncryptedMessageを使用すると、「指定したメッセージが暗号化されています」という意味のフラグ（目印）がつきます。
エクスプローラーやウォレットはそのフラグを参考にメッセージを無用にデコードしなかったり、非表示にしたりなどの処理を行います。
このメソッドが暗号化をするわけではありません。

##### 生データ
```js
sym.RawMessage.create(uint8Arrays[i])
```

#### 最大手数料

ネットワーク手数料については、常に少し多めに払っておけば問題はないのですが、最低限の知識は持っておく必要があります。
アカウントはトランザクションを作成するときに、ここまでは手数料として払ってもいいという最大手数料を指定します。
一方で、ノードはその時々で最も高い手数料となるトランザクションのみブロックにまとめて収穫しようとします。
つまり、多く払ってもいいというトランザクションが他に多く存在すると承認されるまでの時間が長くなります。
逆に、より少なく払いたいというトランザクションが多く存在し、その総額が大きい場合は、設定した最大額に満たない手数料額で送信が実現します。

トランザクションサイズ x feeMultiprilerというもので決定されます。
176バイトだった場合 maxFee を100で設定すると 17600μXYM = 0.0176XYMを手数料として支払うことを許容します。
feeMultiprier = 100として指定する方法とmaxFee = 17600 として指定する方法があります。

##### feeMultiprier = 100として指定する方法
```js
tx = sym.TransferTransaction.create(
  ,,,,
  networkType
).setMaxFee(100);
```

##### maxFee = 17600 として指定する方法
```js
tx = sym.TransferTransaction.create(
  ,,,,
  networkType,
  sym.UInt64.fromUint(17600)
);
```

本書では以後、feeMultiprier = 100として指定する方法で統一して説明します。

## 4.3 署名とアナウンス

作成したトランザクションを秘密鍵で署名して、任意のノードを通じてアナウンスします。

### 署名

In [27]:
signedTx = alice.sign(tx,generationHash);
console.log(JSON.stringify(signedTx,null,2));

{
  "payload": "AE0000000000000090C52C0269B6D2FB21728A17043DB645BB78B218F965C8506C91C8F4EFA987E38BE657C91A63132667620208E274EDA447A2A5F9C9D8611C23D42F341DDA9E0070B0C6BDB2C291F1F30D9DA6CF7D1C6DBF54588285B040ADD762EACD853EE4E90000000001985441F843000000000000B5BA3D660300000098EC10797B167D59E419781125EE36676AA61D9E4F90CDAF0E000000000000000048656C6C6F2053796D626F6C21",
  "hash": "FEDDEDCA1C83A2C48E37CD235BEF5536C96CF3472B0C5D93FF75DFDFB5FDD277",
  "signerPublicKey": "70B0C6BDB2C291F1F30D9DA6CF7D1C6DBF54588285B040ADD762EACD853EE4E9",
  "type": 16724,
  "networkType": 152
}

トランザクションの署名にはAccountクラスとgenerationHash値が必要です。

generationHash
- テストネット
    - 7FCCD304802016BEBBCD342A332F91FF1F3BB5E902988B352697BE245F48E836
- メインネット
    - 57F7DA205008026C776CB6AED843393F04CD458E0AA2D9F1D5F31A402072B2D6

generationHash値はそのブロックチェーンネットワークを一意に識別するための値です。
同じ秘密鍵をもつ他のネットワークに使いまわされないようにそのネットワーク個別のハッシュ値を織り交ぜて署名済みトランザクションを作成します。

### アナウンス

In [28]:
res = await txRepo.announce(signedTx).toPromise();
console.log(res);

```js
> TransactionAnnounceResponse {message: 'packet 9 was pushed to the network via /transactions'}
```

上記のスクリプトのように `packet n was pushed to the network` というレスポンスがあれば、トランザクションはノードに受理されたことになります。
これはトランザクションのフォーマット等に異常が無かった程度の意味しかありません。
Symbolではノードの応答速度を極限に高めるため、トランザクションの内容を検証するまえに受信結果の応答を返し接続を切断します。
レスポンス値はこの情報を受け取ったにすぎません。フォーマットに異常があった場合は以下のようなメッセージ応答があります。

##### アナウンスに失敗した場合の応答例
```js
Uncaught Error: {"statusCode":409,"statusMessage":"Unknown Error","body":"{\"code\":\"InvalidArgument\",\"message\":\"payload has an invalid format\"}"}
```

## 4.4 確認


### ステータスの確認

ノードに受理されたトランザクションのステータスを確認

In [10]:
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(signedTx.hash).toPromise();
console.log(JSON.stringify(transactionStatus,null,2));

{
  "group": "confirmed",
  "hash": "8BAD4B9E21212A5C1F5F3BA5F77CC0EE0AFCE5783D880DE788C14626E2A4E439",
  "deadline": {
    "adjustedValue": 14599021233
  },
  "code": "Success",
  "height": {
    "lower": 391307,
    "higher": 0
  }
}

###### 出力例
```js
> TransactionStatus
    group: "confirmed"
    code: "Success"
    deadline: Deadline {adjustedValue: 11989512431}
    hash: "661360E61C37E156B0BE18E52C9F3ED1022DCE846A4609D72DF9FA8A5B667747"
    height: undefined
```

承認されると ` group: "confirmed"`となっています。

受理されたものの、エラーが発生していた場合は以下のような出力となります。トランザクションを書き直して再度アナウンスしてみてください。

```js
> TransactionStatus
    group: "failed"
    code: "Failure_Core_Insufficient_Balance"
    deadline: Deadline {adjustedValue: 11990156766}
    hash: "A82507C6C46DF444E36AC94391EA2D0D7DD1A218948DED465A7A4F9D1B53CA0E"
    height: undefined
```

以下のようにResourceNotFoundエラーが発生した場合はトランザクションが受理されていません。
```js
Uncaught Error: {"statusCode":404,"statusMessage":"Unknown Error","body":"{\"code\":\"ResourceNotFound\",\"message\":\"no resource exists with id '18AEBC9866CD1C15270F18738D577CB1BD4B2DF3EFB28F270B528E3FE583F42D'\"}"}
```

考えられる可能性としては、トランザクションで指定した最大手数料が、ノードで設定された最低手数料に満たない場合や、
アグリゲートトランザクションとしてアナウンスすることが求められているトランザクションを単体のトランザクションでアナウンスした場合に発生するようです。

### 承認確認

トランザクションがブロックに承認されるまでに30秒程度かかります。

#### エクスプローラーで確認
signedTx.hash で取得できるハッシュ値を使ってエクスプローラーで検索してみましょう。


In [13]:
console.log(signedTx.hash);
console.log(`https://testnet.symbol.fyi/transactions/${signedTx.hash}`)

8BAD4B9E21212A5C1F5F3BA5F77CC0EE0AFCE5783D880DE788C14626E2A4E439

https://testnet.symbol.fyi/transactions/8BAD4B9E21212A5C1F5F3BA5F77CC0EE0AFCE5783D880DE788C14626E2A4E439

```js
> "661360E61C37E156B0BE18E52C9F3ED1022DCE846A4609D72DF9FA8A5B667747"
```

- メインネット　
  - https://symbol.fyi/transactions/661360E61C37E156B0BE18E52C9F3ED1022DCE846A4609D72DF9FA8A5B667747
- テストネット　
  - https://testnet.symbol.fyi/transactions/661360E61C37E156B0BE18E52C9F3ED1022DCE846A4609D72DF9FA8A5B667747

#### SDKで確認

In [70]:
txInfo = await txRepo.getTransaction(signedTx.hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(JSON.stringify(txInfo,null,2));
console.log(JSON.stringify(txInfo.transactionInfo,null,2));

{
  "transaction": {
    "type": 16724,
    "network": 152,
    "version": 1,
    "maxFee": "17400",
    "deadline": "14600223413",
    "signature": "90C52C0269B6D2FB21728A17043DB645BB78B218F965C8506C91C8F4EFA987E38BE657C91A63132667620208E274EDA447A2A5F9C9D8611C23D42F341DDA9E00",
    "signerPublicKey": "70B0C6BDB2C291F1F30D9DA6CF7D1C6DBF54588285B040ADD762EACD853EE4E9",
    "recipientAddress": {
      "address": "TDWBA6L3CZ6VTZAZPAISL3RWM5VKMHM6J6IM3LY",
      "networkType": 152
    },
    "mosaics": [],
    "message": "0048656C6C6F2053796D626F6C21"
  }
}

{
  "height": {
    "lower": 391338,
    "higher": 0
  },
  "index": 2,
  "id": "643EE54557F5E76C38318DBC",
  "timestamp": {
    "lower": 1708155896,
    "higher": 3
  },
  "feeMultiplier": 100,
  "hash": "FEDDEDCA1C83A2C48E37CD235BEF5536C96CF3472B0C5D93FF75DFDFB5FDD277",
  "merkleComponentHash": "FEDDEDCA1C83A2C48E37CD235BEF5536C96CF3472B0C5D93FF75DFDFB5FDD277"
}

###### 出力例
```js
> TransferTransaction
    deadline: Deadline {adjustedValue: 12883929118}
    maxFee: UInt64 {lower: 17400, higher: 0}
    message: PlainMessage {type: 0, payload: 'Hello Symbol!'}
    mosaics: []
    networkType: 152
    payloadSize: 174
    recipientAddress: Address {address: 'TDWBA6L3CZ6VTZAZPAISL3RWM5VKMHM6J6IM3LY', networkType: 152}
    signature: "7A3562DCD7FEE4EE9CB456E48EFEEC687647119DC053DE63581FD46CA9D16A829FA421B39179AABBF4DE0C1D987B58490E3F95C37327358E6E461832E3B3A60D"
    signer: PublicAccount {publicKey: '0E5C72B0D5946C1EFEE7E5317C5985F106B739BB0BC07E4F9A288417B3CD6D26', address: Address}
  > transactionInfo: TransactionInfo
        hash: "DA4B672E68E6561EAE560FB89B144AFE1EF75D2BE0D9B6755D90388F8BCC4709"
        height: UInt64 {lower: 330012, higher: 0}
        id: "626413050A21EB5CD286E17D"
        index: 1
        merkleComponentHash: "DA4B672E68E6561EAE560FB89B144AFE1EF75D2BE0D9B6755D90388F8BCC4709"
    type: 16724
    version: 1
```
##### 注意点

トランザクションはブロックで承認されたとしても、ロールバックが発生するとトランザクションの承認が取り消される場合があります。
ブロックが承認された後、数ブロックの承認が進むと、ロールバックの発生する確率は減少していきます。
また、Votingノードの投票で実施されるファイナライズブロックを待つことで、記録されたデータは確実なものとなります。

##### スクリプト例
トランザクションをアナウンスした後は以下のようなスクリプトを流すと、チェーンの状態を把握しやすくて便利です。

In [71]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);

## 4.5トランザクション履歴

Aliceが送受信したトランザクション履歴を一覧で取得します。

In [72]:
result = await txRepo.search(
    {
      group:sym.TransactionGroup.Confirmed,
      embedded:true,
      address:alice.address
    }
  ).toPromise();
  txes = result.data;
  txes.forEach(tx => {
    console.log(JSON.stringify(tx,null,2));
    console.log(JSON.stringify(tx.transactionInfo,null,2));
  })

{
  "transaction": {
    "type": 16724,
    "network": 152,
    "version": 1,
    "maxFee": "200000000",
    "deadline": "14598730739",
    "signature": "0C7AD8B9C88C704B3FE60D65BBA7D1568BB11DA19A16B0E8B5A8A34729F6CFC973455B045B21ACA9566E2B7FFADFCE09152322F33AEB399FC2AB8499F7F34809",
    "signerPublicKey": "81EA7C15E7EC06261C9F654F54EAC4748CFCF00E09A8FE47779ACD14A7602004",
    "recipientAddress": {
      "address": "TAOP6YRVWUH6VMBAZASNLYHL2SEU7DLALXJN7EY",
      "networkType": 152
    },
    "mosaics": [
      {
        "amount": "1000000000",
        "id": "72C0212E67A08BCE"
      }
    ]
  }
}

{
  "height": {
    "lower": 391290,
    "higher": 0
  },
  "index": 0,
  "id": "643EDF5457F5E76C38318D4E",
  "timestamp": {
    "lower": 1706635488,
    "higher": 3
  },
  "feeMultiplier": 1136363,
  "hash": "20C75E64F95E1216D5B9A500F47E22B1D765704BB481AD46C609C099F8ECCED4",
  "merkleComponentHash": "20C75E64F95E1216D5B9A500F47E22B1D765704BB481AD46C609C099F8ECCED4"
}

{
  "transaction": {
    "type": 16724,
    "network": 152,
    "version": 1,
    "maxFee": "17400",
    "deadline": "14599021231",
    "signature": "4A49D829D12DF1AF9B0991679DB7D5457A8FB8074A32229DD449405F29E1F53828FE19D66FB469606B8E3692D11B080FD4A12CD13D143602C3DC47AB2A3F600B",
    "signerPublicKey": "70B0C6BDB2C291F1F30D9DA6CF7D1C6DBF54588285B040ADD762EACD853EE4E9",
    "recipientAddress": {
      "address": "TDWBA6L3CZ6VTZAZPAISL3RWM5VKMHM6J6IM3LY",
      "networkType": 152
    },
    "mosaics": [],
    "message": "0048656C6C6F2053796D626F6C21"
  }
}

{
  "height": {
    "lower": 391307,
    "higher": 0
  },
  "index": 0,
  "id": "643EE17B57F5E76C38318D75",
  "timestamp": {
    "lower": 1707186467,
    "higher": 3
  },
  "feeMultiplier": 100,
  "hash": "C58BF13FAE8D02D090D8D16FADAAFC922C264DCD801477F966505F3B074B0BBB",
  "merkleComponentHash": "C58BF13FAE8D02D090D8D16FADAAFC922C264DCD801477F966505F3B074B0BBB"
}

{
  "transaction": {
    "type": 16724,
    "network": 152,
    "version": 1,
    "maxFee": "17400",
    "deadline": "14599021225",
    "signature": "57C6205A8012320468BBCFD84036707195C42A944F3A70DC8D40E42D56D870733B6E8CDD04ABB3E4B07D2F1879A8BA138021EF2683479CB1CF371866BDCD850D",
    "signerPublicKey": "70B0C6BDB2C291F1F30D9DA6CF7D1C6DBF54588285B040ADD762EACD853EE4E9",
    "recipientAddress": {
      "address": "TDWBA6L3CZ6VTZAZPAISL3RWM5VKMHM6J6IM3LY",
      "networkType": 152
    },
    "mosaics": [],
    "message": "0048656C6C6F2053796D626F6C21"
  }
}

...

{
  "height": {
    "lower": 391338,
    "higher": 0
  },
  "index": 2,
  "id": "643EE54557F5E76C38318DBC",
  "timestamp": {
    "lower": 1708155896,
    "higher": 3
  },
  "feeMultiplier": 100,
  "hash": "FEDDEDCA1C83A2C48E37CD235BEF5536C96CF3472B0C5D93FF75DFDFB5FDD277",
  "merkleComponentHash": "FEDDEDCA1C83A2C48E37CD235BEF5536C96CF3472B0C5D93FF75DFDFB5FDD277"
}

{
  "transaction": {
    "type": 16724,
    "network": 152,
    "version": 1,
    "maxFee": "17400",
    "deadline": "14600223410",
    "signature": "9CEE263AEDFDA1510A9A1B1962288AEB17953AAD3B59A870B4F03C7CA9E5C1608C38076B0E94B806ECD99E86D8ECEE83560837CDA448AAC89F3B65D98A333707",
    "signerPublicKey": "70B0C6BDB2C291F1F30D9DA6CF7D1C6DBF54588285B040ADD762EACD853EE4E9",
    "recipientAddress": {
      "address": "TDWBA6L3CZ6VTZAZPAISL3RWM5VKMHM6J6IM3LY",
      "networkType": 152
    },
    "mosaics": [],
    "message": "0048656C6C6F2053796D626F6C21"
  }
}

{
  "height": {
    "lower": 391338,
    "higher": 0
  },
  "index": 1,
  "id": "643EE54557F5E76C38318DBD",
  "timestamp": {
    "lower": 1708155896,
    "higher": 3
  },
  "feeMultiplier": 100,
  "hash": "D5567D727874923B365E942258E71E503F8E2A913C3FF87977EDF0F6083F50AD",
  "merkleComponentHash": "D5567D727874923B365E942258E71E503F8E2A913C3FF87977EDF0F6083F50AD"
}

{
  "transaction": {
    "type": 16724,
    "network": 152,
    "version": 1,
    "maxFee": "17400",
    "deadline": "14600223412",
    "signature": "FADA535AA6D2837DCEBE75E0AD549127B5000655CCE109AB35396C1FFD7E77CCD070D180D18378BAA1943406DCA4ED10000630FABD1EA38674C2EA54697B7F00",
    "signerPublicKey": "70B0C6BDB2C291F1F30D9DA6CF7D1C6DBF54588285B040ADD762EACD853EE4E9",
    "recipientAddress": {
      "address": "TDWBA6L3CZ6VTZAZPAISL3RWM5VKMHM6J6IM3LY",
      "networkType": 152
    },
    "mosaics": [],
    "message": "0048656C6C6F2053796D626F6C21"
  }
}

{
  "height": {
    "lower": 391338,
    "higher": 0
  },
  "index": 3,
  "id": "643EE54557F5E76C38318DBE",
  "timestamp": {
    "lower": 1708155896,
    "higher": 3
  },
  "feeMultiplier": 100,
  "hash": "68AF5F1FD827737ADF00BC52F48718BC0EFD6549B7C4475D366EE9675793F7BB",
  "merkleComponentHash": "68AF5F1FD827737ADF00BC52F48718BC0EFD6549B7C4475D366EE9675793F7BB"
}

###### 出力例
```js
> TransferTransaction
    type: 16724
    networkType: 152
    payloadSize: 176
    deadline: Deadline {adjustedValue: 11905303680}
    maxFee: UInt64 {lower: 200000000, higher: 0}
    recipientAddress: Address {address: 'TBXUTAX6O6EUVPB6X7OBNX6UUXBMPPAFX7KE5TQ', networkType: 152}
    signature: "E5924A1EB653240A7220405A4DD4E221E71E43327B3BA691D267326FEE3F57458E8721907188DB33A3F2A9CB1D0293845B4D0F1D7A93C8A3389262D1603C7108"
    signer: PublicAccount {publicKey: 'BDFAF3B090270920A30460AA943F9D8D4FCFF6741C2CB58798DBF7A2ED6B75AB', address: Address}
  > message: RawMessage
      payload: ""
      type: -1
  > mosaics: Array(1)
      0: Mosaic
        amount: UInt64 {lower: 10000000, higher: 0}
        id: MosaicId
          id: Id {lower: 760461000, higher: 981735131}
  > transactionInfo: TransactionInfo
      hash: "308472D34BE1A58B15A83B9684278010F2D69B59E39127518BE38A4D22EEF31D"
      height: UInt64 {lower: 301717, higher: 0}
      id: "6255242053E0E706653116F9"
      index: 0
      merkleComponentHash: "308472D34BE1A58B15A83B9684278010F2D69B59E39127518BE38A4D22EEF31D"
```

TransactionTypeは以下の通りです。
```js
{0: 'RESERVED', 16705: 'AGGREGATE_COMPLETE', 16707: 'VOTING_KEY_LINK', 16708: 'ACCOUNT_METADATA', 16712: 'HASH_LOCK', 16716: 'ACCOUNT_KEY_LINK', 16717: 'MOSAIC_DEFINITION', 16718: 'NAMESPACE_REGISTRATION', 16720: 'ACCOUNT_ADDRESS_RESTRICTION', 16721: 'MOSAIC_GLOBAL_RESTRICTION', 16722: 'SECRET_LOCK', 16724: 'TRANSFER', 16725: 'MULTISIG_ACCOUNT_MODIFICATION', 16961: 'AGGREGATE_BONDED', 16963: 'VRF_KEY_LINK', 16964: 'MOSAIC_METADATA', 16972: 'NODE_KEY_LINK', 16973: 'MOSAIC_SUPPLY_CHANGE', 16974: 'ADDRESS_ALIAS', 16976: 'ACCOUNT_MOSAIC_RESTRICTION', 16977: 'MOSAIC_ADDRESS_RESTRICTION', 16978: 'SECRET_PROOF', 17220: 'NAMESPACE_METADATA', 17229: 'MOSAIC_SUPPLY_REVOCATION', 17230: 'MOSAIC_ALIAS', 17232: 'ACCOUNT_OPERATION_RESTRICTION'
```
MessageTypeは以下の通りです。
```js
{0: 'PlainMessage', 1: 'EncryptedMessage', 254: 'PersistentHarvestingDelegationMessage', -1: 'RawMessage'}
```
## 4.6 アグリゲートトランザクション
Symbolでは複数のトランザクションを1ブロックにまとめてアナウンスすることができます。
最大で100件のトランザクションをまとめることができます（連署者が異なる場合は25アカウントまでを連署指定可能）。
以降の章で扱う内容にアグリゲートトランザクションへの理解が必要な機能が含まれますので、
本章ではアグリゲートトランザクションのうち、簡単なものだけを紹介します。
### 起案者の署名だけが必要な場合

In [73]:
bob = sym.Account.generateNewAccount(networkType);
carol = sym.Account.generateNewAccount(networkType);
innerTx1 = sym.TransferTransaction.create(
    undefined, //Deadline
    bob.address,  //送信先
    [],
    sym.PlainMessage.create("tx1"),
    networkType
);
innerTx2 = sym.TransferTransaction.create(
    undefined, //Deadline
    carol.address,  //送信先
    [],
    sym.PlainMessage.create("tx2"),
    networkType
);
aggregateTx = sym.AggregateTransaction.createComplete(
    sym.Deadline.create(epochAdjustment),
    [
      innerTx1.toAggregate(alice.publicAccount), //送信元アカウントの公開鍵
      innerTx2.toAggregate(alice.publicAccount)  //送信元アカウントの公開鍵
    ],
    networkType,
    [],
    sym.UInt64.fromUint(1000000)
);
signedTx = alice.sign(aggregateTx,generationHash);
await txRepo.announce(signedTx).toPromise();

まず、アグリゲートトランザクションに含めるトランザクションを作成します。
このときDeadlineを指定する必要はありません。
リスト化するときに、生成したトランザクションにtoAggregateを追加して送信元アカウントの公開鍵を指定します。
ちなみに送信元アカウントと署名アカウントが **必ずしも一致するとは限りません** 。
後の章での解説で「Bobの送信トランザクションをAliceが署名する」といった事が起こり得るためこのような書き方をします。
これはSymbolブロックチェーンでトランザクションを扱ううえで最も重要な概念になります。
なお、本章で扱うトランザクションは同じAliceですので、アグリゲートボンデッドトランザクションへの署名もAliceを指定します。

<span style="color:red">

### 補足　TransactionsStatusの確認</span>  

In [77]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

https://testnet.symbol.fyi/transactions/3232322700491BA3F9FF49D29F8D73478EC97E3E4487C8545AB2328889CE3537

## 4.7 現場で使えるヒント
### 存在証明
アカウントの章でアカウントによるデータの署名と検証する方法について説明しました。
このデータをトランザクションに載せてブロックチェーンが承認することで、
アカウントがある時刻にあるデータの存在を認知したことを消すことができなくなります。
タイムスタンプの刻印された電子署名を利害関係者間で所有することと同じ意味があると考えることもできます。
（法律的な判断は他の方にお任せします）
ブロックチェーンは、この消せない「アカウントが認知したという事実」の存在をもって送信などのデータ更新を行います。
また、誰もがまだ知らないはずの事実を知っていたことの証明としてブロックチェーンを利用することもできます。
ここでは、その存在が証明されたデータをトランザクションに載せる２つの方法について説明します。


![image](https://user-images.githubusercontent.com/47712051/232884314-f691a99c-0f8e-4c0f-bf73-c9de24ecfbdf.png)

#### デジタルデータのハッシュ値(SHA256)出力方法
ファイルの要約値をブロックチェーンに記録することでそのファイルの存在を証明することができます。
各OSごとのファイルのSHA256でハッシュ値を計算する方法は下記の通りです。
```sh
#Windows
certutil -hashfile WINファイルパス SHA256
#Mac
shasum -a 256 MACファイルパス
#Linux
sha256sum Linuxファイルパス
```

<span style="color:red">

### 補足　ハッシュの作成とブロックチェーンへの記録</span>  

実際にファイルのハッシュ値を作成しブロックチェーンに記録することで、存在証明を行ないます。  
画面下部のターミナルから以下コマンドを実行し、ipynbファイルのハッシュ値を取得します。

```sh
#Windows
certutil -hashfile trainingCamp/assets/quick_learning_symbol_04.image.png SHA256
#Mac
shasum -a 256 trainingCamp/assets/quick_learning_symbol_04.image.png
```

###### 出力例
```sh
6113cc7d8619a4fc27aafffd9fbc3cacdd19bfecda0b311240435616d3a724fb  trainingCamp/assets/quick_learning_symbol_04.image.png
```

出力されたハッシュ値をTxのメッセージとしてalice自身に送ることで、その時点でのファイルの存在証明を残す事ができます。

In [79]:
tx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment), //Deadline:有効期限
    alice.address,  //自分自身に転送
    [],
    sym.PlainMessage.create("ハッシュ値  ファイル名"), //🌟ここに出力されたハッシュ値とファイル名を入力
    networkType //テストネット・メインネット区分
).setMaxFee(100); //手数料
signedTx = alice.sign(tx,generationHash);
console.log(JSON.stringify(signedTx,null,2));
res = await txRepo.announce(signedTx).toPromise();
console.log(res);

{
  "payload": "C100000000000000DCDFB9B65D8E72D0D4ECFD79491A060A578179358D278F9FB53641990E7F6A51A882D315A24707533E975B3B56ECEFDC014F3BE3F3DFC19B85E9F6FD79DCD40370B0C6BDB2C291F1F30D9DA6CF7D1C6DBF54588285B040ADD762EACD853EE4E90000000001985441644B0000000000005721736603000000981CFF6235B50FEAB020C824D5E0EBD4894F8D605DD2DF93210000000000000000E3838FE38383E382B7E383A5E580A42020E38395E382A1E382A4E383ABE5908D",
  "hash": "525457BF42E39A0F4F0B3DC018BC23F0D080B4EA19321D8D0994A71CEB2E8100",
  "signerPublicKey": "70B0C6BDB2C291F1F30D9DA6CF7D1C6DBF54588285B040ADD762EACD853EE4E9",
  "type": 16724,
  "networkType": 152
}

TransactionのStatusを確認します

In [80]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

https://testnet.symbol.fyi/transactions/525457BF42E39A0F4F0B3DC018BC23F0D080B4EA19321D8D0994A71CEB2E8100

![image](https://user-images.githubusercontent.com/47712051/232884294-13877a30-e09d-48d4-8630-f52eafd00121.png)

#### 大きなデータの分割
トランザクションのペイロードには1023バイトしか格納できないため、
大きなデータは分割してペイロードに詰め込んでアグリゲートトランザクションにします。
```js
bigdata = 'C00200000000000093B0B985101C1BDD1BC2BF30D72F35E34265B3F381ECA464733E147A4F0A6B9353547E2E08189EF37E50D271BEB5F09B81CE5816BB34A153D2268520AF630A0A0E5C72B0D5946C1EFEE7E5317C5985F106B739BB0BC07E4F9A288417B3CD6D26000000000198414140770200000000002A769FB40000000076B455CFAE2CCDA9C282BF8556D3E9C9C0DE18B0CBE6660ACCF86EB54AC51B33B001000000000000DB000000000000000E5C72B0D5946C1EFEE7E5317C5985F106B739BB0BC07E4F9A288417B3CD6D26000000000198544198205C1A4CE06C45B3A896B1B2360E03633B9F36BF7F22338B000000000000000066653465353435393833444430383935303645394533424446434235313637433046394232384135344536463032413837364535303734423641303337414643414233303344383841303630353343353345354235413835323835443639434132364235343233343032364244444331443133343139464435353438323930334242453038423832304100000000006800000000000000B2D4FD84B2B63A96AA37C35FC6E0A2341CEC1FD19C8FFC8D93CCCA2B028D1E9D000000000198444198205C1A4CE06C45B3A896B1B2360E03633B9F36BF7F2233BC089179EBBE01A81400140035383435344434373631364336433635373237396800000000000000B2D4FD84B2B63A96AA37C35FC6E0A2341CEC1FD19C8FFC8D93CCCA2B028D1E9D000000000198444198205C1A4CE06C45B3A896B1B2360E03633B9F36BF7F223345ECB996EDDB9BEB1400140035383435344434373631364336433635373237390000000000000000B2D4FD84B2B63A96AA37C35FC6E0A2341CEC1FD19C8FFC8D93CCCA2B028D1E9D5A71EBA9C924EFA146897BE6C9BB3DACEFA26A07D687AC4A83C9B03087640E2D1DDAE952E9DDBC33312E2C8D021B4CC0435852C0756B1EBD983FCE221A981D02';
let payloads = [];
for (let i = 0; i < bigdata.length / 1023; i++) {
    payloads.push(bigdata.substr(i * 1023, 1023));
}
console.log(payloads);
```


<span style="color:red">

### 補足　画像のBASE64化とブロックチェーンへの記録</span>  